## Set Up Environment

In [ ]:
# Cell 0

network = "custom"  # set this to one of mainnet, yeouido, euljiro, pagoda, or custom
contracts_file_custom = 'contracts_20210124194550.pkl'

connections = {
"mainnet": {"iconservice": "https://ctz.solidwallet.io",       "nid": 1},
"yeouido": {"iconservice": "https://bicon.net.solidwallet.io", "nid": 3},
"euljiro": {"iconservice": "https://test-ctz.solidwallet.io",  "nid": 2},
"pagoda":  {"iconservice": "https://zicon.net.solidwallet.io", "nid":80},
"custom":  {"iconservice": "http://18.144.108.38:9000",        "nid": 3}}

env = connections[network]

In [ ]:
# Cell 1

from iconsdk.exception import JSONRPCException
from iconsdk.libs.in_memory_zip import gen_deploy_data_content
from iconsdk.icon_service import IconService
from iconsdk.providers.http_provider import HTTPProvider
from iconsdk.builder.transaction_builder import CallTransactionBuilder, TransactionBuilder, DeployTransactionBuilder
from iconsdk.builder.call_builder import CallBuilder
from iconsdk.signed_transaction import SignedTransaction
from iconsdk.wallet.wallet import KeyWallet
from iconsdk.utils.convert_type import convert_hex_str_to_int
from repeater import retry
from shutil import make_archive
import pickle as pkl
from datetime import datetime
from time import sleep
import json
import os

ICX = 1000000000000000000 # 10**18
GOVERNANCE_ADDRESS = "cx0000000000000000000000000000000000000000"
ORACLE = "cx61a36e5d10412e03c907a507d1e8c6c3856d9964"

@retry(JSONRPCException, tries=10, delay=1, back_off=2)
def get_tx_result(_tx_hash):
    tx_result = icon_service.get_transaction_result(_tx_hash)
    return tx_result

In [ ]:
# Cell 2

icon_service = IconService(HTTPProvider(env["iconservice"], 3))
NID = env["nid"]

In [ ]:
# Cell 3

wallet = KeyWallet.load("keystores/keystore_test1.json", "test1_Account")
# Balanced test wallet
with open("keystores/balanced_test.pwd", "r") as f:
    key_data = f.read()
btest_wallet = KeyWallet.load("keystores/balanced_test.json", key_data)
# wallet = btest_wallet

### Send ICX to Balanced test wallet

In [ ]:
# Cell 4
# Only necessary if running locally or on the private tbears server.

test_wallet = KeyWallet.load("./keystores/keystore_test1.json", "test1_Account")
transaction = TransactionBuilder()\
    .from_(wallet.get_address())\
    .to(btest_wallet.get_address())\
    .value(1000000 * ICX)\
    .step_limit(1000000) \
    .nid(NID) \
    .nonce(2) \
    .version(3) \
    .build()
signed_transaction = SignedTransaction(transaction, test_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

### Deployed to testnet

In [ ]:
# Cell 5
# The following addresses are those deployed to the testnet at the start of the audit.

# Uncomment these lines and comment out the definition below to load contracts from a saved file.
# with open(contracts_file_custom, 'rb') as f:
#     contracts = pkl.load(f)

contracts = {'loans': {'zip': 'core_contracts/loans.zip',
  'SCORE': 'cxcd9ae763fabcf6c3c9a475f727264e9cfffdfaf2'},
 'staking': {'zip': 'core_contracts/staking.zip',
  'SCORE': 'cx52aab6b1dd316396e684573464af36c634059721'},
 'dividends': {'zip': 'core_contracts/dividends.zip',
  'SCORE': 'cx02bba98bf78c91329c6958c61ceed1e7bad3b68a'},
 'reserve': {'zip': 'core_contracts/reserve_fund.zip',
  'SCORE': 'cx418f4a6392b88568cb9587c6ca638b358f8ca728'},
 'rewards': {'zip': 'core_contracts/rewards.zip',
  'SCORE': 'cx65fb9d6c9bafdc9cdbe6bf258fd31b90f2f4ab8d'},
 'dex': {'zip': 'core_contracts/dex.zip',
  'SCORE': 'cx291dacbb875a94b364194a5febaac4e6318681f7'},
 'governance': {'zip': 'core_contracts/governance.zip',
  'SCORE': 'cx4179247372b8ce4c9edd8013dd3918852bd0c7fa'},
 'dummy_oracle': {'zip': 'core_contracts/dummy_oracle.zip',
  'SCORE': 'cx61a36e5d10412e03c907a507d1e8c6c3856d9964'},
 'sicx': {'zip': 'token_contracts/sicx.zip',
  'SCORE': 'cx5b0d6a5b244e9254ca05592389672c5185d9f5f8'},
 'icd': {'zip': 'token_contracts/icd.zip',
  'SCORE': 'cx699694fa8dbb76d904d33b63f90dee7cf0440e71'},
 'bal': {'zip': 'token_contracts/bal.zip',
  'SCORE': 'cxd89295b894b72b3b5966848f6fe2bbb997f99e71'},
 'bwt': {'zip': 'token_contracts/bwt.zip',
  'SCORE': 'cx6fda38cfc5c828763306ad7fb4225a8062bd0aee'}}

In [ ]:
# New testnet deployment, 2/19/2021

# staking = 'cxadd9557a07d4aab925dd04b7a6da4ea1d67bef9c'
# sicx = 'cxb7eda227d9ed9c6fc2a74aa5dd7e9a3913f6c281'

contracts = {'loans': {'zip': 'core_contracts/loans.zip',
  'SCORE': 'cx1082087c37b11cb4f25204c1d187263301e75754'},
 'staking': {'zip': 'core_contracts/staking.zip',
  'SCORE': 'cxfe0399264cd6a49fa6c62291034490f874f36e98'},
 'dividends': {'zip': 'core_contracts/dividends.zip',
  'SCORE': 'cxa269e34bd9597f65f0017535565b1385f572fa77'},
 'reserve': {'zip': 'core_contracts/reserve.zip',
  'SCORE': 'cx8813806e90778117cb8f97589f5b3d3972c57046'},
 'rewards': {'zip': 'core_contracts/rewards.zip',
  'SCORE': 'cxe35e1d194d352abc167c0e7a7202f8b47d3e8571'},
 'dex': {'zip': 'core_contracts/dex.zip',
  'SCORE': 'cxe7a760b0d865b9f8bdeae1d7756a317ddd348fe8'},
 'governance': {'zip': 'core_contracts/governance.zip',
  'SCORE': 'cx9e015cc5038661eccd574cb55413b3749a92acb3'},
 'dummy_oracle': {'zip': 'core_contracts/dummy_oracle.zip',
  'SCORE': 'cx61a36e5d10412e03c907a507d1e8c6c3856d9964'},
 'sicx': {'zip': 'token_contracts/sicx.zip',
  'SCORE': 'cx04827a21e04dbfb13db9df4a64c57a6378251559'},
 'icd': {'zip': 'token_contracts/icd.zip',
  'SCORE': 'cxfa4e28608ab3d959a6f87be112feec6f4055130a'},
 'bal': {'zip': 'token_contracts/bal.zip',
  'SCORE': 'cxb256c7b7c8a6a17a21e21bcb772edef7071810be'},
 'bwt': {'zip': 'token_contracts/bwt.zip',
  'SCORE': 'cxb863d14950a9b9ad00049bec61ab752f57cfd750'}}


In [ ]:
# Private tbears server - with 1 hour day.

contracts = {'loans': {'zip': 'core_contracts/loans.zip',
  'SCORE': 'cx3c3fe6eeb1f69601edaf2de000ae1bff62f3dace'},
 'staking': {'zip': 'core_contracts/staking.zip',
  'SCORE': 'cxdc140c1d6a4ba79037b76adff8b94970cdde2f25'},
 'dividends': {'zip': 'core_contracts/dividends.zip',
  'SCORE': 'cx354c0ce44e543ebce9478bf0de3621c95473fc90'},
 'reserve': {'zip': 'core_contracts/reserve.zip',
  'SCORE': 'cx0a478572e443bbfd66a35d124f63a2db9b637483'},
 'rewards': {'zip': 'core_contracts/rewards.zip',
  'SCORE': 'cxe6074249f58898af2d6d00859ecb54e3f658ebe5'},
 'dex': {'zip': 'core_contracts/dex.zip',
  'SCORE': 'cx4c73961fd85f13e822f72c2c96120c99e2e637d5'},
 'governance': {'zip': 'core_contracts/governance.zip',
  'SCORE': 'cxeb7eb8d592ce76d0fcf1a3edd56b50ad2f1f72c5'},
 'dummy_oracle': {'zip': 'core_contracts/dummy_oracle.zip',
  'SCORE': 'cx355d0e49525a681723ac0df9347189b5bb7814c8'},
 'sicx': {'zip': 'token_contracts/sicx.zip',
  'SCORE': 'cxc60ef756b875786c9f820313ac4015e45726b47d'},
 'icd': {'zip': 'token_contracts/icd.zip',
  'SCORE': 'cx14a00b375511d6bb1ad4e2c8eee41f4c8a167095'},
 'bal': {'zip': 'token_contracts/bal.zip',
  'SCORE': 'cxbc2752abceba2bf179c1bfe0bc3f88caedcf2bd9'},
 'bwt': {'zip': 'token_contracts/bwt.zip',
  'SCORE': 'cx0c6bb52e5e34ad8848de911be8ae52be38992363'}}

In [ ]:
{"loans": "cx1bdfc395e3c9ae3ea4a4efa7287367c59c3e24fb",
 "staking": "cxdc140c1d6a4ba79037b76adff8b94970cdde2f25",
 "dividends": "cxdebb4f8f9e3d6c705acbfc878e748250daa1d687",
 "reserve": "cx17d4ac46533ac046ed0d96637b4e1676ceed3f94",
 "rewards": "cxe8bdae7e809fd1fa6447b0246e5f47e5afc74f13",
 "dex": "cxb07a58e85729ce51ff89dda2cade64f40c679d01",
 "governance": "cx23258bfc634b3d6f2e87cb68c8dcc55342c2c53b",
 "dummy_oracle": "cx355d0e49525a681723ac0df9347189b5bb7814c8",
 "sicx": "cxc60ef756b875786c9f820313ac4015e45726b47d",
 "icd": "cxb5e629e5ffc84f55835505030c996e5f13865f6a",
 "bal": "cx9e119a716f99360c516228240ab14ae5dd314da2",
 "bwt": "cxf56ba1e3e3a8c198e8acf88cd450faf2facfdc06"}

In [841]:
# Cell 6

deploy = list(contracts.keys())[:]
deploy.remove('dummy_oracle')
deploy.remove('staking')
deploy.remove('sicx')
for directory in {"core_contracts", "token_contracts"}:
    with os.scandir(directory) as it:
        for file in it:
            archive_name = directory + "/" + file.name
            if file.is_dir() and file.name in deploy:
                make_archive(archive_name, "zip", directory, file.name)
                contracts[file.name]['zip'] = archive_name + '.zip'

contracts

{'loans': {'zip': 'core_contracts/loans.zip',
  'SCORE': 'cx13cf0e3449d5e7f362bc5cfc31cefb11009f79df'},
 'staking': {'zip': 'core_contracts/staking.zip',
  'SCORE': 'cxdc140c1d6a4ba79037b76adff8b94970cdde2f25'},
 'dividends': {'zip': 'core_contracts/dividends.zip',
  'SCORE': 'cxb359edcfd74fd8b74dbe15ac1082121a9eceeda6'},
 'reserve': {'zip': 'core_contracts/reserve.zip',
  'SCORE': 'cxe140658fc108c53dce21211e2e9fab5af00e3f24'},
 'rewards': {'zip': 'core_contracts/rewards.zip',
  'SCORE': 'cx8b95fca41e267a2f4e91a839b7c4b513b99c7d2c'},
 'dex': {'zip': 'core_contracts/dex.zip',
  'SCORE': 'cx20002ffedf9ffa2227e90c031a2ff032a7508023'},
 'governance': {'zip': 'core_contracts/governance.zip',
  'SCORE': 'cxe6eaab2bb8bf77f5e0842b573aab07fe5a7a2a04'},
 'dummy_oracle': {'zip': 'core_contracts/dummy_oracle.zip',
  'SCORE': 'cx355d0e49525a681723ac0df9347189b5bb7814c8'},
 'sicx': {'zip': 'token_contracts/sicx.zip',
  'SCORE': 'cxc60ef756b875786c9f820313ac4015e45726b47d'},
 'icd': {'zip': 'token_co

## Deploy All SCOREs

In [830]:
# Cell 7
# prints out contracts dictionary with all new contract addresses

step_limit = 4000100000

for score in deploy:
    loc = contracts[score]
    deploy_transaction = DeployTransactionBuilder()\
        .from_(wallet.get_address())\
        .to(GOVERNANCE_ADDRESS)\
        .nid(NID)\
        .nonce(100)\
        .content_type("application/zip")\
        .content(gen_deploy_data_content(loc["zip"]))\
        .build()

#     step_limit = icon_service.estimate_step(deploy_transaction) + 100000
    
    signed_transaction = SignedTransaction(deploy_transaction, wallet, step_limit)
    tx_hash = icon_service.send_transaction(signed_transaction)

    res = get_tx_result(tx_hash)
    contracts[score]["SCORE"] = res.get('scoreAddress', '')
    print(score)
    print(f'Status: {res["status"]}')
    if len(res["eventLogs"]) > 0:
        for item in res["eventLogs"]:
            print(f'{item} \n')
    if res['status'] == 0:
        print(f'Failure: {res["failure"]}')
    print('\n')

contracts

Retrying in 1 seconds...
Retrying in 2 seconds...
loans
Status: 1


Retrying in 1 seconds...
dividends
Status: 1


Retrying in 1 seconds...
Retrying in 2 seconds...
reserve
Status: 1


Retrying in 1 seconds...
rewards
Status: 1


Retrying in 1 seconds...
Retrying in 2 seconds...
dex
Status: 1


Retrying in 1 seconds...
Retrying in 2 seconds...
governance
Status: 1


Retrying in 1 seconds...
icd
Status: 1


Retrying in 1 seconds...
Retrying in 2 seconds...
bal
Status: 1


Retrying in 1 seconds...
bwt
Status: 1




{'loans': {'zip': 'core_contracts/loans.zip',
  'SCORE': 'cx13cf0e3449d5e7f362bc5cfc31cefb11009f79df'},
 'staking': {'zip': 'core_contracts/staking.zip',
  'SCORE': 'cxdc140c1d6a4ba79037b76adff8b94970cdde2f25'},
 'dividends': {'zip': 'core_contracts/dividends.zip',
  'SCORE': 'cxb359edcfd74fd8b74dbe15ac1082121a9eceeda6'},
 'reserve': {'zip': 'core_contracts/reserve.zip',
  'SCORE': 'cxe140658fc108c53dce21211e2e9fab5af00e3f24'},
 'rewards': {'zip': 'core_contracts/rewards.zip',
  'SCORE': 'cx8b95fca41e267a2f4e91a839b7c4b513b99c7d2c'},
 'dex': {'zip': 'core_contracts/dex.zip',
  'SCORE': 'cx20002ffedf9ffa2227e90c031a2ff032a7508023'},
 'governance': {'zip': 'core_contracts/governance.zip',
  'SCORE': 'cxe6eaab2bb8bf77f5e0842b573aab07fe5a7a2a04'},
 'dummy_oracle': {'zip': 'core_contracts/dummy_oracle.zip',
  'SCORE': 'cx355d0e49525a681723ac0df9347189b5bb7814c8'},
 'sicx': {'zip': 'token_contracts/sicx.zip',
  'SCORE': 'cxc60ef756b875786c9f820313ac4015e45726b47d'},
 'icd': {'zip': 'token_co

In [ ]:
# Cell 8
# saves contracts dictionary to file

# now = datetime.utcnow()
# filename = f'contracts_{now.year}{str(now.month).zfill(2)}{str(now.day).zfill(2)}{str(now.hour).zfill(2)}{str(now.minute).zfill(2)}{str(now.second).zfill(2)}.pkl'
# print(f'saving contracts list as {filename}')
# with open(filename, 'wb') as f:
#     pkl.dump(contracts, f)

In [753]:
# Cell 9
# Prints out dictionary of SCORE addresses for use in testing UI

def get_scores_json(contracts):
    scores = {}
    for score in contracts:
        scores[score] = contracts[score]['SCORE']
    return json.dumps(scores)

print(get_scores_json(contracts))

{"loans": "cx3c3fe6eeb1f69601edaf2de000ae1bff62f3dace", "staking": "cxdc140c1d6a4ba79037b76adff8b94970cdde2f25", "dividends": "cx354c0ce44e543ebce9478bf0de3621c95473fc90", "reserve": "cx0a478572e443bbfd66a35d124f63a2db9b637483", "rewards": "cxe6074249f58898af2d6d00859ecb54e3f658ebe5", "dex": "cx4c73961fd85f13e822f72c2c96120c99e2e637d5", "governance": "cxeb7eb8d592ce76d0fcf1a3edd56b50ad2f1f72c5", "dummy_oracle": "cx355d0e49525a681723ac0df9347189b5bb7814c8", "sicx": "cxc60ef756b875786c9f820313ac4015e45726b47d", "icd": "cx14a00b375511d6bb1ad4e2c8eee41f4c8a167095", "bal": "cxbc2752abceba2bf179c1bfe0bc3f88caedcf2bd9", "bwt": "cx0c6bb52e5e34ad8848de911be8ae52be38992363"}


### Deploy a single SCORE

In [ ]:
# Cell 10

# Deploy a single SCORE
contract = 'bwt'
zip_file = contracts[contract]['zip']
deploy_transaction = DeployTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(GOVERNANCE_ADDRESS)\
    .nid(NID)\
    .nonce(100)\
    .content_type("application/zip")\
    .content(gen_deploy_data_content(zip_file))\
    .build()

step_limit = 10000000000


signed_transaction = SignedTransaction(deploy_transaction, wallet, step_limit)
tx_hash = icon_service.send_transaction(signed_transaction)

res = get_tx_result(tx_hash)
contracts[contract]["SCORE"] = res.get('scoreAddress', '')
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

## Update a SCORE

In [842]:
# Cell 11

contract = 'governance'
update = contracts[contract]['SCORE']
zip_file = contracts[contract]['zip']
deploy_transaction = DeployTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(update)\
    .nid(NID)\
    .nonce(100)\
    .content_type("application/zip")\
    .content(gen_deploy_data_content(zip_file))\
    .build()

# step_limit = icon_service.estimate_step(deploy_transaction) + 100000
step_limit = 3000000000

signed_transaction = SignedTransaction(deploy_transaction, wallet, step_limit)
tx_hash = icon_service.send_transaction(signed_transaction)

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

Retrying in 1 seconds...
Status: 1


## Set parameters

In [843]:
# Cell 12

addresses = {'loans': contracts['loans']['SCORE'],
             'dex': contracts['dex']['SCORE'],
             'staking': contracts['staking']['SCORE'],
             'rewards': contracts['rewards']['SCORE'],
             'reserve': contracts['reserve']['SCORE'],
             'dividends': contracts['dividends']['SCORE'],
             'oracle': contracts['dummy_oracle']['SCORE'],
             'sicx': contracts['sicx']['SCORE'],
             'icd': contracts['icd']['SCORE'],
             'bal': contracts['bal']['SCORE'],
             'bwt': contracts['bwt']['SCORE']
            }

settings = [{'contract': 'sicx', 'method': 'setAdmin', 'params':{'_admin': contracts['staking']['SCORE']}},
            {'contract': 'sicx', 'method': 'setStakingAddress', 'params':{'_address': contracts['staking']['SCORE']}},
            {'contract': 'staking', 'method': 'setSicxAddress', 'params':{'_address': contracts['sicx']['SCORE']}},
            {'contract': 'icd', 'method': 'setGovernance', 'params':{'_address': contracts['governance']['SCORE']}},
            {'contract': 'bal', 'method': 'setGovernance', 'params':{'_address': contracts['governance']['SCORE']}},
            {'contract': 'bwt', 'method': 'setGovernance', 'params':{'_address': contracts['governance']['SCORE']}},
            {'contract': 'dex', 'method': 'setGovernance', 'params':{'_address': contracts['governance']['SCORE']}},
            {'contract': 'reserve', 'method': 'setGovernance', 'params':{'_address': contracts['governance']['SCORE']}},
            {'contract': 'rewards', 'method': 'setGovernance', 'params':{'_address': contracts['governance']['SCORE']}},
            {'contract': 'dividends', 'method': 'setGovernance', 'params':{'_address': contracts['governance']['SCORE']}},
            {'contract': 'loans', 'method': 'setGovernance', 'params':{'_address': contracts['governance']['SCORE']}},
            {'contract': 'governance', 'method': 'setAddresses', 'params':{'_addresses': addresses}},
            {'contract': 'governance', 'method': 'launchBalanced', 'params':{}}]


In [844]:
# Cell 13

results = {}
for sett in settings:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(wallet.get_address())\
        .to(contracts[sett['contract']]['SCORE'])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)

    res = get_tx_result(tx_hash)
    results[f'{sett["contract"]}|{sett["method"]}|{sett["params"]}'] = res
    print(f'Status: {res["status"]}')
    if len(res["eventLogs"]) > 0:
        for item in res["eventLogs"]:
            print(f'{item} \n')
    if res['status'] == 0:
        print(f'Failure: {res["failure"]}')



Calling setAdmin, with parameters {'_admin': 'cxdc140c1d6a4ba79037b76adff8b94970cdde2f25'} on the sicx contract.
Retrying in 1 seconds...
Retrying in 2 seconds...
Status: 1


Calling setStakingAddress, with parameters {'_address': 'cxdc140c1d6a4ba79037b76adff8b94970cdde2f25'} on the sicx contract.
Retrying in 1 seconds...
Retrying in 2 seconds...
Status: 1


Calling setSicxAddress, with parameters {'_address': 'cxc60ef756b875786c9f820313ac4015e45726b47d'} on the staking contract.
Retrying in 1 seconds...
Status: 1


Calling setGovernance, with parameters {'_address': 'cxe6eaab2bb8bf77f5e0842b573aab07fe5a7a2a04'} on the icd contract.
Retrying in 1 seconds...
Retrying in 2 seconds...
Status: 1


Calling setGovernance, with parameters {'_address': 'cxe6eaab2bb8bf77f5e0842b573aab07fe5a7a2a04'} on the bal contract.
Retrying in 1 seconds...
Status: 1


Calling setGovernance, with parameters {'_address': 'cxe6eaab2bb8bf77f5e0842b573aab07fe5a7a2a04'} on the bwt contract.
Retrying in 1 second

## Run through read-only methods on all SCOREs

In [ ]:
# Cell 14

ro_methods = {"loans": ["name", "getAssetTokens", "getCollateralTokens", "getTotalCollateral", "getAvailableAssets", "assetCount", "getSnapshot"],
              "staking": ["name", "getSicxAddress", "getTodayRate"],
              "dividends": ["name", "getBalances"],
              "reserve": ["name", "getBalances"],
              "rewards": ["name", "getDataSourceNames"],
              "governance": ["name", "getAddresses"],
              "sicx": ["name", "symbol", "decimals", "totalSupply", "getAdmin", "lastPriceInLoop"],
              "icd": ["name", "symbol", "decimals", "totalSupply", "getAdmin"],
              "bal": ["name", "symbol", "decimals", "totalSupply", "getAdmin"]
              }

In [ ]:
# Cell 15
# Runs through the readonly methods defined above. These can also be viewed using the testing UI

results = {}
for score in ro_methods.keys():
    print()
    print(f'SCORE: {score}')
    print(f'Address: {contracts[score]["SCORE"]}')
    results[score] = {}
    for method in ro_methods[score]:
        call = CallBuilder().from_(wallet.get_address())\
                            .to(contracts[score]['SCORE'])\
                            .method(method)\
                            .build()
        result = icon_service.call(call)
        if method in ['totalSupply', 'decimals', 'get_total_collateral', 'get_rate']:
            result = int(result, 0)
            if method in ['totalSupply', 'get_total_collateral', 'get_rate']:
                result = result / ICX
        print(f'{method}: {result}')
        results[score][method] = result

### Get sICX from Staking

In [ ]:
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['staking']['SCORE'])\
    .value(800 * ICX)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("addCollateral")\
    .params({})\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

## Deposit ICX and Mint ICD Loan

In [845]:
# Cell 16

data1 = "{\"method\": \"_deposit_and_borrow\", \"params\": {\"_sender\": \"".encode("utf-8")
data2 = "\", \"_asset\": \"ICD\", \"_amount\": 40000000000000000000}}".encode("utf-8")
params = {'_data1': data1, '_data2': data2}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['loans']['SCORE'])\
    .value(800 * ICX)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("addCollateral")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

Retrying in 1 seconds...
Status: 1
{'scoreAddress': 'cx13cf0e3449d5e7f362bc5cfc31cefb11009f79df', 'indexed': ['ICXTransfer(Address,Address,int)', 'cx13cf0e3449d5e7f362bc5cfc31cefb11009f79df', 'cxdc140c1d6a4ba79037b76adff8b94970cdde2f25', '0x2b5e3af16b18800000'], 'data': []} 

{'scoreAddress': 'cx0000000000000000000000000000000000000000', 'indexed': ['IScoreClaimedV2(Address,int,int)', 'cxdc140c1d6a4ba79037b76adff8b94970cdde2f25'], 'data': ['0x4574cdc55c4e97d0e9b0', '0x11c7e2c1e261f19dee']} 

{'scoreAddress': 'cxc60ef756b875786c9f820313ac4015e45726b47d', 'indexed': ['Transfer(Address,Address,int,bytes)', 'cxc60ef756b875786c9f820313ac4015e45726b47d', 'hxe7af5fcfd8dfc67530a01a0e403882687528dfcb', '0x2b1490f85467663bdb'], 'data': ['0x7b226d6574686f64223a20225f6465706f7369745f616e645f626f72726f77222c2022706172616d73223a207b225f73656e646572223a2022687865376166356663666438646663363735333061303161306534303338383236383735323864666362222c20225f6173736574223a2022494344222c20225f616d6f756e74223a20

### Just deposit collateral

In [846]:
# Cell 17

data1 = "{\"method\": \"_deposit_and_borrow\", \"params\": {\"_sender\": \"".encode("utf-8")
data2 = "\", \"_asset\": \"\", \"_amount\": 0}}".encode("utf-8")
params = {'_data1': data1, '_data2': data2}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['loans']['SCORE'])\
    .value(30 * ICX)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("addCollateral")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

Retrying in 1 seconds...
Retrying in 2 seconds...
Status: 1
{'scoreAddress': 'cx13cf0e3449d5e7f362bc5cfc31cefb11009f79df', 'indexed': ['ICXTransfer(Address,Address,int)', 'cx13cf0e3449d5e7f362bc5cfc31cefb11009f79df', 'cxdc140c1d6a4ba79037b76adff8b94970cdde2f25', '0x1a055690d9db80000'], 'data': []} 

{'scoreAddress': 'cxc60ef756b875786c9f820313ac4015e45726b47d', 'indexed': ['Transfer(Address,Address,int,bytes)', 'cxc60ef756b875786c9f820313ac4015e45726b47d', 'hxe7af5fcfd8dfc67530a01a0e403882687528dfcb', '0x19d1a0b1065755fac'], 'data': ['0x7b226d6574686f64223a20225f6465706f7369745f616e645f626f72726f77222c2022706172616d73223a207b225f73656e646572223a2022687865376166356663666438646663363735333061303161306534303338383236383735323864666362222c20225f6173736574223a2022222c20225f616d6f756e74223a20307d7d']} 

{'scoreAddress': 'cxc60ef756b875786c9f820313ac4015e45726b47d', 'indexed': ['Mint(Address,int,bytes)', 'hxe7af5fcfd8dfc67530a01a0e403882687528dfcb'], 'data': ['0x19d1a0b1065755fac', '0x7b226d6

### Repay Some ICD

In [847]:
# Cell 18

data = "{\"method\": \"_repay_loan\", \"params\": {}}".encode("utf-8")
params = {'_to': contracts['loans']['SCORE'], '_value': 5 * ICX, '_data': data}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['icd']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

Retrying in 1 seconds...
Retrying in 2 seconds...
Status: 1
{'scoreAddress': 'cx8d566120272b78d37b9d6cba34a239b655e55ed1', 'indexed': ['Transfer(Address,Address,int,bytes)', 'cx13cf0e3449d5e7f362bc5cfc31cefb11009f79df', 'cx8d566120272b78d37b9d6cba34a239b655e55ed1', '0x4563918244f40000'], 'data': ['0x4e6f6e65']} 

{'scoreAddress': 'cx8d566120272b78d37b9d6cba34a239b655e55ed1', 'indexed': ['Burn(Address,int)', 'cx13cf0e3449d5e7f362bc5cfc31cefb11009f79df'], 'data': ['0x4563918244f40000']} 

{'scoreAddress': 'cx13cf0e3449d5e7f362bc5cfc31cefb11009f79df', 'indexed': ['LoanRepaid(Address,str,int,str)', 'hxe7af5fcfd8dfc67530a01a0e403882687528dfcb', 'ICD', '0x4563918244f40000'], 'data': ['Loan of 5.0 ICD repaid to Balanced.']} 

{'scoreAddress': 'cx8d566120272b78d37b9d6cba34a239b655e55ed1', 'indexed': ['Transfer(Address,Address,int,bytes)', 'hxe7af5fcfd8dfc67530a01a0e403882687528dfcb', 'cx13cf0e3449d5e7f362bc5cfc31cefb11009f79df', '0x4563918244f40000'], 'data': ['0x7b226d6574686f64223a20225f7265

### Withdraw Some Collateral

In [848]:
# Cell 19

params = {'_value': 25 * ICX}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['loans']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("withdrawCollateral")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

Retrying in 1 seconds...
Status: 1
{'scoreAddress': 'cxc60ef756b875786c9f820313ac4015e45726b47d', 'indexed': ['Transfer(Address,Address,int,bytes)', 'cx13cf0e3449d5e7f362bc5cfc31cefb11009f79df', 'hxe7af5fcfd8dfc67530a01a0e403882687528dfcb', '0x15af1d78b58c40000'], 'data': ['0x4e6f6e65']} 

{'scoreAddress': 'cx13cf0e3449d5e7f362bc5cfc31cefb11009f79df', 'indexed': ['TokenTransfer(Address,int,str)', 'hxe7af5fcfd8dfc67530a01a0e403882687528dfcb', '0x15af1d78b58c40000'], 'data': ['Collateral withdrawn. 25000000000000000000 sICX sent to hxe7af5fcfd8dfc67530a01a0e403882687528dfcb.']} 



### Add collateral directly

In [849]:
data1 = "{\"method\": \"_deposit_and_borrow\", \"params\": {\"_sender\": \"".encode("utf-8")
data2 = "\", \"_asset\": \"\", \"_amount\": 0}}".encode("utf-8")
params = {'_to': contracts['loans']['SCORE'], '_value': 20 * ICX, '_data': data1 + b'hxe7af5fcfd8dfc67530a01a0e403882687528dfcb' + data2}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['sicx']['SCORE'])\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

Retrying in 1 seconds...
Status: 1
{'scoreAddress': 'cx13cf0e3449d5e7f362bc5cfc31cefb11009f79df', 'indexed': ['CollateralReceived(Address,str)', 'hxe7af5fcfd8dfc67530a01a0e403882687528dfcb', 'sICX'], 'data': []} 

{'scoreAddress': 'cxc60ef756b875786c9f820313ac4015e45726b47d', 'indexed': ['Transfer(Address,Address,int,bytes)', 'hxe7af5fcfd8dfc67530a01a0e403882687528dfcb', 'cx13cf0e3449d5e7f362bc5cfc31cefb11009f79df', '0x1158e460913d00000'], 'data': ['0x7b226d6574686f64223a20225f6465706f7369745f616e645f626f72726f77222c2022706172616d73223a207b225f73656e646572223a2022687865376166356663666438646663363735333061303161306534303338383236383735323864666362222c20225f6173736574223a2022222c20225f616d6f756e74223a20307d7d']} 



### Test Liquidation

In [797]:
# Cell 20
# 1. Deposit collateral to fresh wallet

data1 = "{\"method\": \"_deposit_and_borrow\", \"params\": {\"_sender\": \"".encode("utf-8")
data2 = "\", \"_asset\": \"\", \"_amount\": 0}}".encode("utf-8")
params = {'_data1': data1, '_data2': data2}
transaction = CallTransactionBuilder()\
    .from_(btest_wallet.get_address())\
    .to(contracts['loans']['SCORE'])\
    .value(782769 * ICX // 1000)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("addCollateral")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, btest_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

Retrying in 1 seconds...
Retrying in 2 seconds...
Status: 1
{'scoreAddress': 'cx3c3fe6eeb1f69601edaf2de000ae1bff62f3dace', 'indexed': ['ICXTransfer(Address,Address,int)', 'cx3c3fe6eeb1f69601edaf2de000ae1bff62f3dace', 'cxdc140c1d6a4ba79037b76adff8b94970cdde2f25', '0x2a6f1a22364bbe8000'], 'data': []} 

{'scoreAddress': 'cxc60ef756b875786c9f820313ac4015e45726b47d', 'indexed': ['Transfer(Address,Address,int,bytes)', 'cxc60ef756b875786c9f820313ac4015e45726b47d', 'hx3f01840a599da07b0f620eeae7aa9c574169a4be', '0x2a2706567a5b05449b'], 'data': ['0x7b226d6574686f64223a20225f6465706f7369745f616e645f626f72726f77222c2022706172616d73223a207b225f73656e646572223a2022687833663031383430613539396461303762306636323065656165376161396335373431363961346265222c20225f6173736574223a2022222c20225f616d6f756e74223a20307d7d']} 

{'scoreAddress': 'cxc60ef756b875786c9f820313ac4015e45726b47d', 'indexed': ['Mint(Address,int,bytes)', 'hx3f01840a599da07b0f620eeae7aa9c574169a4be'], 'data': ['0x2a2706567a5b05449b', '0x7b22

In [798]:
# Cell 21
# 2. Check the account standing after depositing collateral - Should be 'No Debt', and
#    holding '0x2a6f1a22364bbe8000' worth of sICX.

params = {'_owner': btest_wallet.get_address()}
call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['loans']['SCORE'])\
                    .method('getAccountPositions')\
                    .params(params)\
                    .build()
result = icon_service.call(call)
result

{'pos_id': '0x66',
 'created': '0x5bc20dc3ae659',
 'address': 'hx3f01840a599da07b0f620eeae7aa9c574169a4be',
 'snap_id': '0x3',
 'snaps_length': '0x1',
 'last_snap': '0x3',
 'first day': '0x3',
 'assets': {'sICX': '0x2a2706567a5b05449b'},
 'replay_index': '0x3c',
 'events_behind': '0x0',
 'total_debt': '0x0',
 'ratio': '0x0',
 'standing': 'No Debt'}

#### 3. Update the loans SCORE to add a loan of ICD to the Balanced test wallet
Code in the on_update() method will mint and add the ICD. Adds an undercollateralized amount of ICD, 200 ICD.

In [805]:
# Cell 22
# 3a. May need to run this a few times to get the position below liquidate threshold.

contract = 'loans'
update = contracts[contract]['SCORE']
zip_file = contracts[contract]['zip']
deploy_transaction = DeployTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(update)\
    .nid(NID)\
    .nonce(100)\
    .content_type("application/zip")\
    .content(gen_deploy_data_content(zip_file))\
    .build()

# step_limit = icon_service.estimate_step(deploy_transaction) + 100000
step_limit = 3000000000

signed_transaction = SignedTransaction(deploy_transaction, wallet, step_limit)
tx_hash = icon_service.send_transaction(signed_transaction)

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

Retrying in 1 seconds...
Status: 1


In [800]:
# Cell 23
# 4. Check the new standing of the account. Should have added '0xad78ebc5ac6200000' ICD and have standing of 'Liquidate'.

params = {'_owner': btest_wallet.get_address()}
call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['loans']['SCORE'])\
                    .method('getAccountPositions')\
                    .params(params)\
                    .build()
result = icon_service.call(call)
result

{'pos_id': '0x66',
 'created': '0x5bc20dc3ae659',
 'address': 'hx3f01840a599da07b0f620eeae7aa9c574169a4be',
 'snap_id': '0x3',
 'snaps_length': '0x1',
 'last_snap': '0x3',
 'first day': '0x3',
 'assets': {'sICX': '0x2a2706567a5b05449b', 'ICD': '0xad78ebc5ac6200000'},
 'replay_index': '0x3c',
 'events_behind': '0x0',
 'total_debt': '0x1c62496c6bb1edb218',
 'ratio': '0x14bf4efe169d20b0',
 'standing': 'Liquidate'}

In [ ]:
# Cell 24
# 5. Replay events to update standing for this position.

address = wallets[10].get_address()
params = {'_owner': address} # wallet.get_address()}
transaction = CallTransactionBuilder()\
    .from_(address)\
    .to(contracts['loans']['SCORE'])\
    .value(0)\
    .step_limit(100000000)\
    .nid(NID)\
    .nonce(100)\
    .method("updateStanding")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallets[10])
tx_hash = icon_service.send_transaction(signed_transaction)

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

In [ ]:
params = {'_owner': btest_wallet.get_address()}
call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['loans']['SCORE'])\
                    .method('hasDebt')\
                    .params(params)\
                    .build()
result = icon_service.call(call)
result

In [ ]:
# Cell 25
# 6. Check the new standing of the account. Should have added '0xad78ebc5ac6200000' ICD and have standing of 'Liquidate'.

params = {'_owner': btest_wallet.get_address()}
call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['loans']['SCORE'])\
                    .method('getAccountPositions')\
                    .params(params)\
                    .build()
result = icon_service.call(call)
result

In [806]:
# Cell 24
# 5. Liquidate the account position.

params = {'_owner': btest_wallet.get_address()}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['loans']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("liquidate")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

Retrying in 1 seconds...
Status: 1
{'scoreAddress': 'cx14a00b375511d6bb1ad4e2c8eee41f4c8a167095', 'indexed': ['OraclePrice(str,str,Address,int)', 'USDICX', 'BandChain', 'cx355d0e49525a681723ac0df9347189b5bb7814c8'], 'data': ['0x2454d8dcb2cf44bb']} 

{'scoreAddress': 'cx3c3fe6eeb1f69601edaf2de000ae1bff62f3dace', 'indexed': ['Snapshot(int)', '0x4'], 'data': []} 

{'scoreAddress': 'cx3c3fe6eeb1f69601edaf2de000ae1bff62f3dace', 'indexed': ['PositionStanding(Address,str,str,str)', 'hx3f01840a599da07b0f620eeae7aa9c574169a4be', 'Liquidate'], 'data': ['1.49500045451835', 'Position up to date.']} 

{'scoreAddress': 'cxc60ef756b875786c9f820313ac4015e45726b47d', 'indexed': ['Transfer(Address,Address,int,bytes)', 'cx3c3fe6eeb1f69601edaf2de000ae1bff62f3dace', 'hxe7af5fcfd8dfc67530a01a0e403882687528dfcb', '0x484cc4028266ded8'], 'data': ['0x4e6f6e65']} 

{'scoreAddress': 'cx3c3fe6eeb1f69601edaf2de000ae1bff62f3dace', 'indexed': ['TokenTransfer(Address,int,str)', 'hxe7af5fcfd8dfc67530a01a0e403882687528d

In [802]:
# Cell 25
# 6. Check the standing of the account after liquidation. Should now have zero balance for sICX and ICD.

params = {'_owner': btest_wallet.get_address()}
call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['loans']['SCORE'])\
                    .method('getAccountPositions')\
                    .params(params)\
                    .build()
result = icon_service.call(call)
result

{'pos_id': '0x66',
 'created': '0x5bc20dc3ae659',
 'address': 'hx3f01840a599da07b0f620eeae7aa9c574169a4be',
 'snap_id': '0x3',
 'snaps_length': '0x1',
 'last_snap': '0x3',
 'first day': '0x3',
 'assets': {'sICX': '0x2a2706567a5b05449b', 'ICD': '0xad78ebc5ac6200000'},
 'replay_index': '0x3c',
 'events_behind': '0x0',
 'total_debt': '0x1c62496c6bb1edb218',
 'ratio': '0x14bf4efe169d20b0',
 'standing': 'Liquidate'}

In [ ]:
# Create a set of wallets

# wallets = []
# for i in range(200):
#     wallets.append(KeyWallet.create())
# with open("../wallets.pkl", 'wb') as f:
#     pkl.dump(wallets, f)

In [ ]:
with open("../wallets.pkl", 'rb') as f:
    wallets = pkl.load(f)

In [761]:
for i in range(200):
    print(f'{i:3d}: {icon_service.get_balance(wallets[i].get_address()) / 10**18: 9.3f}')

  0:  24881.808
  1:  24931.784
  2:  24441.772
  3:  24681.822
  4:  24681.829
  5:  24681.786
  6:  24681.827
  7:  24681.826
  8:  24681.760
  9:  24681.486
 10:  24761.173
 11:  24761.800
 12:  24761.848
 13:  24761.848
 14:  24761.845
 15:  24761.843
 16:  24761.848
 17:  24761.847
 18:  24761.847
 19:  24717.807
 20:  26141.898
 21:  26141.898
 22:  26141.899
 23:  26141.899
 24:  26141.900
 25:  26141.900
 26:  26141.898
 27:  26141.894
 28:  26141.895
 29:  26141.894
 30:  26141.890
 31:  26141.901
 32:  26141.902
 33:  26141.902
 34:  26141.902
 35:  26141.903
 36:  26141.902
 37:  26141.902
 38:  26141.903
 39:  26141.902
 40:  26141.903
 41:  26141.902
 42:  26141.902
 43:  26141.902
 44:  26141.901
 45:  26141.902
 46:  26141.901
 47:  26141.900
 48:  26141.900
 49:  26141.900
 50:  26141.903
 51:  26141.902
 52:  26141.902
 53:  26141.903
 54:  26141.902
 55:  26141.902
 56:  26141.902
 57:  26141.903
 58:  26141.905
 59:  26141.903
 60:  26141.897
 61:  26141.899
 62:  26

In [ ]:
# 10000 ICX into each wallet. These wallets are already more than well stocked enough on the tbears server.

for i in range(100):
    transaction = TransactionBuilder()\
        .from_(wallet.get_address())\
        .to(wallets[i].get_address())\
        .value(20 * ICX)\
        .step_limit(1000000) \
        .nid(NID) \
        .nonce(2) \
        .version(3) \
        .build()
    signed_transaction = SignedTransaction(transaction, wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

In [763]:
# Mint some ICD to each wallet

for i in range(0, 100):
    data1 = "{\"method\": \"_deposit_and_borrow\", \"params\": {\"_sender\": \"".encode("utf-8")
    data2 = "\", \"_asset\": \"ICD\", \"_amount\": 50000000000000000000}}".encode("utf-8")
    params = {'_data1': data1, '_data2': data2}
    transaction = CallTransactionBuilder()\
        .from_(wallets[i].get_address())\
        .to(contracts['loans']['SCORE'])\
        .value(800 * ICX)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method("addCollateral")\
        .params(params)\
        .build()
    signed_transaction = SignedTransaction(transaction, wallets[i])
    tx_hash = icon_service.send_transaction(signed_transaction)
    sleep(2)

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

Status: 1
{'scoreAddress': 'cx3c3fe6eeb1f69601edaf2de000ae1bff62f3dace', 'indexed': ['ICXTransfer(Address,Address,int)', 'cx3c3fe6eeb1f69601edaf2de000ae1bff62f3dace', 'cxdc140c1d6a4ba79037b76adff8b94970cdde2f25', '0x2b5e3af16b18800000'], 'data': []} 

{'scoreAddress': 'cxc60ef756b875786c9f820313ac4015e45726b47d', 'indexed': ['Transfer(Address,Address,int,bytes)', 'cxc60ef756b875786c9f820313ac4015e45726b47d', 'hx06fe1b158d82669a578d77b29f699b959f287fe2', '0x2b1490f85467663bdb'], 'data': ['0x7b226d6574686f64223a20225f6465706f7369745f616e645f626f72726f77222c2022706172616d73223a207b225f73656e646572223a2022687830366665316231353864383236363961353738643737623239663639396239353966323837666532222c20225f6173736574223a2022494344222c20225f616d6f756e74223a2035303030303030303030303030303030303030307d7d']} 

{'scoreAddress': 'cxc60ef756b875786c9f820313ac4015e45726b47d', 'indexed': ['Mint(Address,int,bytes)', 'hx06fe1b158d82669a578d77b29f699b959f287fe2'], 'data': ['0x2b1490f85467663bdb', '0x7b226d6574

data1 = "{\"method\": \"_deposit_and_borrow\", \"params\": {\"_sender\": \"".encode("utf-8")
data2 = "\", \"_asset\": \"ICD\", \"_amount\": 5000000000000000000}}".encode("utf-8")
params = {'_data1': data1, '_data2': data2}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['loans']['SCORE'])\
    .value(18 * ICX)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("addCollateral")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
sleep(2)

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

data1 = "{\"method\": \"_deposit_and_borrow\", \"params\": {\"_sender\": \"".encode("utf-8")
data2 = "\", \"_asset\": \"ICD\", \"_amount\": 4000000000000000000}}".encode("utf-8")
params = {'_data1': data1, '_data2': data2}
transaction = CallTransactionBuilder()\
    .from_(wallets[2].get_address())\
    .to(contracts['loans']['SCORE'])\
    .value(80 * ICX)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("addCollateral")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallets[2])
tx_hash = icon_service.send_transaction(signed_transaction)
sleep(2)

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

In [816]:
# Mint some ICD to each wallet

for i in range(50, 150):
    data1 = "{\"method\": \"_deposit_and_borrow\", \"params\": {\"_sender\": \"".encode("utf-8")
    data2 = "\", \"_asset\": \"ICD\", \"_amount\": 20000000000000000000}}".encode("utf-8")
    params = {'_data1': data1, '_data2': data2}
    transaction = CallTransactionBuilder()\
        .from_(wallets[i].get_address())\
        .to(contracts['loans']['SCORE'])\
        .value(500 * ICX)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method("addCollateral")\
        .params(params)\
        .build()
    signed_transaction = SignedTransaction(transaction, wallets[i])
    tx_hash = icon_service.send_transaction(signed_transaction)
    sleep(2)

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

Status: 1
{'scoreAddress': 'cx3c3fe6eeb1f69601edaf2de000ae1bff62f3dace', 'indexed': ['ICXTransfer(Address,Address,int)', 'cx3c3fe6eeb1f69601edaf2de000ae1bff62f3dace', 'cxdc140c1d6a4ba79037b76adff8b94970cdde2f25', '0x1b1ae4d6e2ef500000'], 'data': []} 

{'scoreAddress': 'cxc60ef756b875786c9f820313ac4015e45726b47d', 'indexed': ['Transfer(Address,Address,int,bytes)', 'cxc60ef756b875786c9f820313ac4015e45726b47d', 'hxc7952bbb371fe996ea82141b2891414b07a0e033', '0x1aecda9b34c09fe569'], 'data': ['0x7b226d6574686f64223a20225f6465706f7369745f616e645f626f72726f77222c2022706172616d73223a207b225f73656e646572223a2022687863373935326262623337316665393936656138323134316232383931343134623037613065303333222c20225f6173736574223a2022494344222c20225f616d6f756e74223a2032303030303030303030303030303030303030307d7d']} 

{'scoreAddress': 'cxc60ef756b875786c9f820313ac4015e45726b47d', 'indexed': ['Mint(Address,int,bytes)', 'hxc7952bbb371fe996ea82141b2891414b07a0e033'], 'data': ['0x1aecda9b34c09fe569', '0x7b226d6574

In [820]:
# redeem 20 ICD from each of 60 wallets that do not have positions on Balanced. This will use up all
# of the liquidation pool and require some replay events to be recorded.
# i = 0
for i in range(90, 100):
    params = {'_to': wallets[i+100].get_address(), '_value': 15 * ICX}
    transaction = CallTransactionBuilder()\
        .from_(wallets[i].get_address())\
        .to(contracts['icd']['SCORE'])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method("transfer")\
        .params(params)\
        .build()
    signed_transaction = SignedTransaction(transaction, wallets[i])
    tx_hash = icon_service.send_transaction(signed_transaction)
    sleep(2)
    data = "{\"method\": \"_retire_asset\", \"params\": {}}".encode("utf-8")
    params = {'_to': contracts['loans']['SCORE'], '_value': 15 * ICX, '_data': data}
    transaction = CallTransactionBuilder()\
        .from_(wallets[i+100].get_address())\
        .to(contracts['icd']['SCORE'])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method("transfer")\
        .params(params)\
        .build()
    signed_transaction = SignedTransaction(transaction, wallets[i+100])
    tx_hash = icon_service.send_transaction(signed_transaction)
    sleep(1)

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

Status: 1
{'scoreAddress': 'cx14a00b375511d6bb1ad4e2c8eee41f4c8a167095', 'indexed': ['Transfer(Address,Address,int,bytes)', 'cx3c3fe6eeb1f69601edaf2de000ae1bff62f3dace', 'cx14a00b375511d6bb1ad4e2c8eee41f4c8a167095', '0xcf20406c88b48000'], 'data': ['0x4e6f6e65']} 

{'scoreAddress': 'cx14a00b375511d6bb1ad4e2c8eee41f4c8a167095', 'indexed': ['Burn(Address,int)', 'cx3c3fe6eeb1f69601edaf2de000ae1bff62f3dace'], 'data': ['0xcf20406c88b48000']} 

{'scoreAddress': 'cxc60ef756b875786c9f820313ac4015e45726b47d', 'indexed': ['Transfer(Address,Address,int,bytes)', 'cx3c3fe6eeb1f69601edaf2de000ae1bff62f3dace', 'hx79be478f16567cae3b40ebd3049e117a01a6fd2e', '0x21aa618b4ea3e8fd8'], 'data': ['0x4e6f6e65']} 

{'scoreAddress': 'cx3c3fe6eeb1f69601edaf2de000ae1bff62f3dace', 'indexed': ['TokenTransfer(Address,int,str)', 'hx79be478f16567cae3b40ebd3049e117a01a6fd2e', '0x21aa618b4ea3e8fd8'], 'data': ['Collateral redeemed. 38813737603842346968 sICX sent to hx79be478f16567cae3b40ebd3049e117a01a6fd2e.']} 

{'scoreAd

In [818]:
res

{'txHash': '0xe2346ad3c3f6cbcc4ee358b5ef77cc8021cf05f9329b4a1361c997c0516a0c40',
 'blockHeight': 399892,
 'blockHash': '0xaadae2c476cd06d2633aedc03af4dd15e19eba2d5cb2f571e19816c97c5781a0',
 'txIndex': 1,
 'to': 'cx3c3fe6eeb1f69601edaf2de000ae1bff62f3dace',
 'stepUsed': 870540,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 870540,
 'eventLogs': [{'scoreAddress': 'cx3c3fe6eeb1f69601edaf2de000ae1bff62f3dace',
   'indexed': ['ICXTransfer(Address,Address,int)',
    'cx3c3fe6eeb1f69601edaf2de000ae1bff62f3dace',
    'cxdc140c1d6a4ba79037b76adff8b94970cdde2f25',
    '0x1b1ae4d6e2ef500000'],
   'data': []},
  {'scoreAddress': 'cxc60ef756b875786c9f820313ac4015e45726b47d',
   'indexed': ['Transfer(Address,Address,int,bytes)',
    'cxc60ef756b875786c9f820313ac4015e45726b47d',
    'hxc7952bbb371fe996ea82141b2891414b07a0e033',
    '0x1aecda9b34c09fe569'],
   'data': ['0x7b226d6574686f64223a20225f6465706f7369745f616e645f626f72726f77222c2022706172616d73223a207b225f73656e646572223a20226878633739353

In [815]:
params = {'_owner': 'hx52d7a7b6ded08200cced994bd28f8a844dd7bbdd'}
transaction = CallTransactionBuilder()\
    .from_(wallets[80].get_address())\
    .to(contracts['loans']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("updateStanding")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallets[80])
tx_hash = icon_service.send_transaction(signed_transaction)

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

Retrying in 1 seconds...
Retrying in 2 seconds...
Status: 1
{'scoreAddress': 'cx3c3fe6eeb1f69601edaf2de000ae1bff62f3dace', 'indexed': ['PositionStanding(Address,str,str,str)', 'hx52d7a7b6ded08200cced994bd28f8a844dd7bbdd', 'Mining'], 'data': ['7.600967342647941', 'Position up to date.']} 



In [795]:
for i in range(100):
    params = {'_owner': wallets[i].get_address()}
    transaction = CallTransactionBuilder()\
        .from_(wallets[i].get_address())\
        .to(contracts['loans']['SCORE'])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method("updateStanding")\
        .params(params)\
        .build()
    signed_transaction = SignedTransaction(transaction, wallets[i])
    tx_hash = icon_service.send_transaction(signed_transaction)

    print(i)
    res = get_tx_result(tx_hash)
    print(f'Status: {res["status"]}')
    if len(res["eventLogs"]) > 0:
        for item in res["eventLogs"]:
            print(f'{item} \n')
    if res['status'] == 0:
        print(f'Failure: {res["failure"]}')

0
Retrying in 1 seconds...
Status: 1
{'scoreAddress': 'cx14a00b375511d6bb1ad4e2c8eee41f4c8a167095', 'indexed': ['OraclePrice(str,str,Address,int)', 'USDICX', 'BandChain', 'cx355d0e49525a681723ac0df9347189b5bb7814c8'], 'data': ['0x2454d8dcb2cf44bb']} 

{'scoreAddress': 'cx3c3fe6eeb1f69601edaf2de000ae1bff62f3dace', 'indexed': ['Snapshot(int)', '0x3'], 'data': []} 

{'scoreAddress': 'cx3c3fe6eeb1f69601edaf2de000ae1bff62f3dace', 'indexed': ['PositionStanding(Address,str,str,str)', 'hx52d7a7b6ded08200cced994bd28f8a844dd7bbdd', 'Mining'], 'data': ['7.600967342647941', 'Position up to date.']} 

1
Retrying in 1 seconds...
Retrying in 2 seconds...
Status: 1
{'scoreAddress': 'cxbc2752abceba2bf179c1bfe0bc3f88caedcf2bd9', 'indexed': ['Mint(Address,int,bytes)', 'cxbc2752abceba2bf179c1bfe0bc3f88caedcf2bd9'], 'data': ['0x152d02c7e14af6800000', '0x4e6f6e65']} 

{'scoreAddress': 'cxbc2752abceba2bf179c1bfe0bc3f88caedcf2bd9', 'indexed': ['Transfer(Address,Address,int,bytes)', 'cxbc2752abceba2bf179c1bfe0

Retrying in 2 seconds...
Status: 1
{'scoreAddress': 'cx3c3fe6eeb1f69601edaf2de000ae1bff62f3dace', 'indexed': ['PositionStanding(Address,str,str,str)', 'hxd143c1b2a8893519da0aaa716627b243638be1b0', 'Mining'], 'data': ['7.600967342647941', 'Position up to date.']} 

22
Retrying in 1 seconds...
Status: 1
{'scoreAddress': 'cx3c3fe6eeb1f69601edaf2de000ae1bff62f3dace', 'indexed': ['PositionStanding(Address,str,str,str)', 'hx87ffb28c6e175e4e39d60694261223057c275d39', 'Mining'], 'data': ['7.600967342647941', 'Position up to date.']} 

23
Retrying in 1 seconds...
Retrying in 2 seconds...
Status: 1
{'scoreAddress': 'cx3c3fe6eeb1f69601edaf2de000ae1bff62f3dace', 'indexed': ['PositionStanding(Address,str,str,str)', 'hxd9b7891257e75b586be41d9b2085745eb1bde5c6', 'Mining'], 'data': ['7.600967342647941', 'Position up to date.']} 

24
Retrying in 1 seconds...
Status: 1
{'scoreAddress': 'cx3c3fe6eeb1f69601edaf2de000ae1bff62f3dace', 'indexed': ['PositionStanding(Address,str,str,str)', 'hxbd183e4ed975e3a32

Retrying in 2 seconds...
Status: 1
{'scoreAddress': 'cx3c3fe6eeb1f69601edaf2de000ae1bff62f3dace', 'indexed': ['PositionStanding(Address,str,str,str)', 'hx11d7ddbfe3e55a18ef70ee37d991247516269771', 'Mining'], 'data': ['7.600967342647941', 'Position up to date.']} 

50
Retrying in 1 seconds...
Status: 1
{'scoreAddress': 'cx3c3fe6eeb1f69601edaf2de000ae1bff62f3dace', 'indexed': ['PositionStanding(Address,str,str,str)', 'hx55338217233cd53f349d6629a8bf86cb29955df4', 'Mining'], 'data': ['7.600967342647941', 'Position up to date.']} 

51
Retrying in 1 seconds...
Status: 1
{'scoreAddress': 'cx3c3fe6eeb1f69601edaf2de000ae1bff62f3dace', 'indexed': ['PositionStanding(Address,str,str,str)', 'hx7d4f32df9740b569070cf14eb77c6c9b6bf34004', 'Mining'], 'data': ['7.600967342647941', 'Position up to date.']} 

52
Retrying in 1 seconds...
Retrying in 2 seconds...
Status: 1
{'scoreAddress': 'cx3c3fe6eeb1f69601edaf2de000ae1bff62f3dace', 'indexed': ['PositionStanding(Address,str,str,str)', 'hx5fc65d3311aacf64f

Status: 1
{'scoreAddress': 'cx3c3fe6eeb1f69601edaf2de000ae1bff62f3dace', 'indexed': ['PositionStanding(Address,str,str,str)', 'hxc52233ba6aa1077d79aa6a8209d2d8273fd891a2', 'Mining'], 'data': ['7.600967342647941', 'Position up to date.']} 

78
Retrying in 1 seconds...
Retrying in 2 seconds...
Status: 1
{'scoreAddress': 'cx3c3fe6eeb1f69601edaf2de000ae1bff62f3dace', 'indexed': ['PositionStanding(Address,str,str,str)', 'hx9ac108461a4a0f1c4ffcb76912a748b9f9158314', 'Mining'], 'data': ['7.600967342647941', 'Position up to date.']} 

79
Retrying in 1 seconds...
Status: 1
{'scoreAddress': 'cx3c3fe6eeb1f69601edaf2de000ae1bff62f3dace', 'indexed': ['PositionStanding(Address,str,str,str)', 'hx4b6f0f658013ecc39a67aefa88ccfa1de5e23b3f', 'Mining'], 'data': ['7.600967342647941', 'Position up to date.']} 

80
Retrying in 1 seconds...
Retrying in 2 seconds...
Status: 1
{'scoreAddress': 'cx3c3fe6eeb1f69601edaf2de000ae1bff62f3dace', 'indexed': ['PositionStanding(Address,str,str,str)', 'hx6713cae481e3c708a

In [ ]:
params = {'_holders': [wallet.get_address(), btest_wallet.get_address()]}
call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['rewards']['SCORE'])\
                    .method("getBalnHoldings")\
                    .params(params)\
                    .build()
result = icon_service.call(call)
i = 0
{i: [key, result[key]] for i, key in enumerate(result.keys())}

In [821]:
# Check BALN that has been distributed on the rewards SCORE, and to platform contract addresses.

params = {}
call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['loans']['SCORE'])\
                    .method("borrowerCount")\
                    .params(params)\
                    .build()
borrowerCount = int(icon_service.call(call), 0)

addresses = []
for i in range(1, borrowerCount + 1):
    params = {'_index': i, '_day': -1}
    call = CallBuilder().from_(wallet.get_address())\
                        .to(contracts['loans']['SCORE'])\
                        .method("getPositionByIndex")\
                        .params(params)\
                        .build()
    position = icon_service.call(call)
    addresses.append(position['address'])

params = {'_holders': addresses}
call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['rewards']['SCORE'])\
                    .method("getBalnHoldings")\
                    .params(params)\
                    .build()
holders = icon_service.call(call)

total_balances = 0
baln_balances = {}
for contract in ['rewards', 'reserve', 'bwt']:
    params = {'_owner': contracts[contract]['SCORE']}
    call = CallBuilder().from_(wallet.get_address())\
                        .to(contracts['bal']['SCORE'])\
                        .method("balanceOf")\
                        .params(params)\
                        .build()
    result = int(icon_service.call(call), 0)
    baln_balances[contract] = result / 10**18
    total_balances += result

i = 0
holdings = {i: [key, int(holders[key], 0), int(holders[key], 0) / 10**18] for i, key in enumerate(holders.keys())}
total = 0
for key in holdings:
    total += holdings[key][1]
    print(f'{holdings[key]}')

print(f'Total unclaimed: {total / 10**18}')
print(baln_balances)
print(f'Total BALN: {total_balances / 10**18}')

['hxe7af5fcfd8dfc67530a01a0e403882687528dfcb', 60522082825343139181482, 60522.08282534314]
['hx52d7a7b6ded08200cced994bd28f8a844dd7bbdd', 744779171746568608218, 744.7791717465686]
['hx95bb62ce10924bc26b2ff982df855fdb8c89bb56', 744779171746568608215, 744.7791717465686]
['hxc6077c30d47d9a03f1ad5fcfddb684a1b3fc9d89', 744779171746568608215, 744.7791717465686]
['hx9c434b3eb728c3d8d6817b106384e549ad594c6e', 744779171746568608215, 744.7791717465686]
['hxcdfe37a9d9bc510e4a170183de7f656380c492eb', 744779171746568608215, 744.7791717465686]
['hx1a41f33dc673581ec2491e2405b1bffad0d110d3', 744779171746568608215, 744.7791717465686]
['hx2fdfb52f6204461d2cd7e30fa768dc215c94aa2f', 744779171746568608215, 744.7791717465686]
['hx6b1f5b2571853150a190b16949a83235991e7ad6', 744779171746568607285, 744.7791717465686]
['hxe500ffc615817f52a3173efa09e4eeb0d6e65a3e', 744779171746568607285, 744.7791717465686]
['hx4c554a3373a41f71eae5413e9af4c5faab590265', 744779171746568607285, 744.7791717465686]
['hx916a40705038f79

In [ ]:
params = {}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['rewards']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("claimRewards")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
sleep(2)

print(i)
res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

In [ ]:
for i in range(0, 200):
    params = {}
    transaction = CallTransactionBuilder()\
        .from_(wallets[i].get_address())\
        .to(contracts['rewards']['SCORE'])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method("claimRewards")\
        .params(params)\
        .build()
    signed_transaction = SignedTransaction(transaction, wallets[i])
    tx_hash = icon_service.send_transaction(signed_transaction)
    sleep(2)

    print(i)
    res = get_tx_result(tx_hash)
    print(f'Status: {res["status"]}')
    if len(res["eventLogs"]) > 0:
        for item in res["eventLogs"]:
            print(f'{item} \n')
    if res['status'] == 0:
        print(f'Failure: {res["failure"]}')

In [ ]:
params = {'_name': 'BALNICD', '_snapshot_id': 1, '_limit': 10, '_offset': 0}
call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['loans']['SCORE'])\
                    .method("getDataBatch")\
                    .params(params)\
                    .build()
result = icon_service.call(call)
result

In [ ]:
params = {'_owner': wallet.get_address()}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['loans']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("updateStanding")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

In [ ]:
icon_service.get_balance(wallet.get_address()) / 10**18

In [ ]:
balances = {}
for wallet in wallets[:20]:
    address = wallet.get_address()
    params = {'_owner': address}
    call = CallBuilder().from_(address)\
                        .to(contracts['bal']['SCORE'])\
                        .method('availableBalanceOf')\
                        .params(params)\
                        .build()
    result = icon_service.call(call)
    balances[address] = result

In [ ]:
balances = {}
for wallet in wallets[:20]:
    address = wallet.get_address()
    params = {'_owner': address}
    call = CallBuilder().from_(address)\
                        .to(contracts['sicx']['SCORE'])\
                        .method('balanceOf')\
                        .params(params)\
                        .build()
    result = icon_service.call(call)
    balances[address] = result

### DEX SICX Test

Does the following:

1. Sends ICX to the DEX
2. Checks overall ICX balance
3. Checks withdrawal lock
4. Mints some SICX
5. Trades SICX against ICX

In [765]:
# Cell 26

transaction = TransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['dex']['SCORE'])\
    .value(1 * ICX)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(101)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

Retrying in 1 seconds...
Status: 1
{'scoreAddress': 'cx4c73961fd85f13e822f72c2c96120c99e2e637d5', 'indexed': ['Snapshot(int)', '0x1'], 'data': []} 



In [766]:
# Cell 27

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getICXBalance')\
                    .params({'_address': wallet.get_address()}) \
                    .build()
result = icon_service.call(call)
int(result, 0) / 10**18

1.0

In [767]:
# Cell 28

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getICXWithdrawLock')\
                    .build()
result = icon_service.call(call)
datetime.utcfromtimestamp(int(result, 0) // 10**6).strftime('%Y-%m-%d %H:%M:%S')

'2021-02-25 01:34:37'

In [768]:
# Cell 29

params = {'_to': wallet.get_address()}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['staking']['SCORE'])\
    .value(10 * ICX)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("addCollateral")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

Retrying in 1 seconds...
Status: 1
{'scoreAddress': 'cxc60ef756b875786c9f820313ac4015e45726b47d', 'indexed': ['Transfer(Address,Address,int,bytes)', 'cxc60ef756b875786c9f820313ac4015e45726b47d', 'hxe7af5fcfd8dfc67530a01a0e403882687528dfcb', '0x89db69810e17ad8c'], 'data': ['0x4e6f6e65']} 

{'scoreAddress': 'cxc60ef756b875786c9f820313ac4015e45726b47d', 'indexed': ['Mint(Address,int,bytes)', 'hxe7af5fcfd8dfc67530a01a0e403882687528dfcb'], 'data': ['0x89db69810e17ad8c', '0x4e6f6e65']} 

{'scoreAddress': 'cxc60ef756b875786c9f820313ac4015e45726b47d', 'indexed': ['Transfer(Address,Address,int,bytes)', 'hxe7af5fcfd8dfc67530a01a0e403882687528dfcb', 'hxe7af5fcfd8dfc67530a01a0e403882687528dfcb', '0x89db69810e17ad8c'], 'data': ['0x4e6f6e65']} 

{'scoreAddress': 'cxdc140c1d6a4ba79037b76adff8b94970cdde2f25', 'indexed': ['TokenTransfer(Address,int,str)', 'hxe7af5fcfd8dfc67530a01a0e403882687528dfcb', '0x89db69810e17ad8c'], 'data': ['9.933649406103891 sICX minted to hxe7af5fcfd8dfc67530a01a0e40388268752

In [769]:
# Cell 30

data = "{\"method\": \"_swap_icx\"}".encode("utf-8")
params = {'_to': contracts['dex']['SCORE'], '_value': 2 * ICX, '_data': data}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['sicx']['SCORE'])\
    .value(0)\
    .step_limit(30000000)\
    .nid(NID)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

Retrying in 1 seconds...
Retrying in 2 seconds...
Status: 0
Failure: {'code': 5, 'message': 'Too many external calls'}


In [770]:
# Cell 31

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getICXBalance')\
                    .params({'_address': wallet.get_address()}) \
                    .build()
result = icon_service.call(call)
int(result, 0) / 10**18

1.0

In [771]:
# Cell 32

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getNamedPools')\
                    .build()
result = icon_service.call(call)
print(result)

['SICXICX']


In [772]:
# Cell 33

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getDataBatch')\
                    .params({'_name': 'SICXICX', '_limit': 10, '_snapshot_id': 1}) \
                    .build()
result = icon_service.call(call)
print(result)

{}


In [773]:
# Cell 34

data = "{\"method\": \"_deposit\"}".encode("utf-8")
params = {'_to': contracts['dex']['SCORE'], '_value': 2 * ICX, '_data': data}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['sicx']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

Retrying in 1 seconds...
Retrying in 2 seconds...
Status: 1
{'scoreAddress': 'cx4c73961fd85f13e822f72c2c96120c99e2e637d5', 'indexed': ['Deposit(Address,Address,int)', 'cxc60ef756b875786c9f820313ac4015e45726b47d', 'hxe7af5fcfd8dfc67530a01a0e403882687528dfcb'], 'data': ['0x1bc16d674ec80000']} 

{'scoreAddress': 'cxc60ef756b875786c9f820313ac4015e45726b47d', 'indexed': ['Transfer(Address,Address,int,bytes)', 'hxe7af5fcfd8dfc67530a01a0e403882687528dfcb', 'cx4c73961fd85f13e822f72c2c96120c99e2e637d5', '0x1bc16d674ec80000'], 'data': ['0x7b226d6574686f64223a20225f6465706f736974227d']} 



In [774]:
# Cell 35

data = "{\"method\": \"_deposit\"}".encode("utf-8")
params = {'_to': contracts['dex']['SCORE'], '_value': 2 * ICX, '_data': data}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['icd']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

Retrying in 1 seconds...
Retrying in 2 seconds...
Status: 1
{'scoreAddress': 'cx4c73961fd85f13e822f72c2c96120c99e2e637d5', 'indexed': ['Deposit(Address,Address,int)', 'cx14a00b375511d6bb1ad4e2c8eee41f4c8a167095', 'hxe7af5fcfd8dfc67530a01a0e403882687528dfcb'], 'data': ['0x1bc16d674ec80000']} 

{'scoreAddress': 'cx14a00b375511d6bb1ad4e2c8eee41f4c8a167095', 'indexed': ['Transfer(Address,Address,int,bytes)', 'hxe7af5fcfd8dfc67530a01a0e403882687528dfcb', 'cx4c73961fd85f13e822f72c2c96120c99e2e637d5', '0x1bc16d674ec80000'], 'data': ['0x7b226d6574686f64223a20225f6465706f736974227d']} 



In [775]:
# Cell 36

data = "{\"method\": \"_deposit\"}".encode("utf-8")
params = {'_to': contracts['dex']['SCORE'], '_value': 100 * ICX, '_data': data}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['bal']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

Retrying in 1 seconds...
Status: 0
Failure: {'code': 32, 'message': 'BALN: Out of available balance. Please check staked and total balance'}


In [776]:
# Cell 37

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getDeposit')\
                    .params({'_user': wallet.get_address(), '_tokenAddress': contracts['icd']['SCORE']}) \
                    .build()
result = icon_service.call(call)
int(result, 0) / 10**18

2.0

In [777]:
# Cell 38

transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['dex']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("add")\
    .params({'_baseToken': contracts['sicx']['SCORE'], '_quoteToken': contracts['icd']['SCORE'], '_baseValue': 1 * ICX, '_quoteValue': 1 * ICX}) \
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

Retrying in 1 seconds...
Retrying in 2 seconds...
Status: 1
{'scoreAddress': 'cx4c73961fd85f13e822f72c2c96120c99e2e637d5', 'indexed': ['MarketAdded(int,Address,Address,int,int)', '0x2', 'cxc60ef756b875786c9f820313ac4015e45726b47d', 'cx14a00b375511d6bb1ad4e2c8eee41f4c8a167095'], 'data': ['0xde0b6b3a7640000', '0xde0b6b3a7640000']} 

{'scoreAddress': 'cx4c73961fd85f13e822f72c2c96120c99e2e637d5', 'indexed': ['Add(int,Address,int)', '0x2', 'hxe7af5fcfd8dfc67530a01a0e403882687528dfcb'], 'data': ['0x56bc75e2d63100000']} 

{'scoreAddress': 'cx4c73961fd85f13e822f72c2c96120c99e2e637d5', 'indexed': ['TransferSingle(Address,Address,Address,int,int)', 'hxe7af5fcfd8dfc67530a01a0e403882687528dfcb', 'cxf000000000000000000000000000000000000000', 'hxe7af5fcfd8dfc67530a01a0e403882687528dfcb'], 'data': ['0x2', '0x56bc75e2d63100000']} 



In [778]:
# Cell 39

transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['dex']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("permit")\
    .params({'_pid': 1, '_permission': 1}) \
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

Retrying in 1 seconds...
Status: 0
Failure: {'code': 1, 'message': 'hxe7af5fcfd8dfc67530a01a0e403882687528dfcb'}


In [779]:
# Cell 40

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getInversePrice')\
                    .params({'_pid': 2}) \
                    .build()
result = icon_service.call(call)
int(result, 0) / 10**18

1e-08

In [780]:
# Cell 41

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getPoolTotal')\
                    .params({'_pid': 2, '_token': contracts['icd']['SCORE']}) \
                    .build()
result = icon_service.call(call)
int(result, 0) / 10**18

1.0

In [781]:
# Cell 42

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getPoolId')\
                    .params({'_token1Address': contracts['sicx']['SCORE'], '_token2Address': contracts['icd']['SCORE'] }) \
                    .build()
result = icon_service.call(call)
print("pool w/ tokens: " + str(contracts['sicx']['SCORE']) + ' & ' + str(contracts['icd']['SCORE']))
print(result)

pool w/ tokens: cxc60ef756b875786c9f820313ac4015e45726b47d & cx14a00b375511d6bb1ad4e2c8eee41f4c8a167095
0x2


In [782]:
# Cell 43

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getPoolBase')\
                    .params({'_pid': 2}) \
                    .build()
result = icon_service.call(call)
print(result)

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getPoolQuote')\
                    .params({'_pid': 2}) \
                    .build()
result = icon_service.call(call)
print(result)

cxc60ef756b875786c9f820313ac4015e45726b47d
cx14a00b375511d6bb1ad4e2c8eee41f4c8a167095


In [783]:
# Cell 44

transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['dex']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("setMarketName")\
    .params({'_pid':2, '_name': 'SICXICD'}) \
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')
print('\n')

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getNamedPools')\
                    .build()
result = icon_service.call(call)
print(result)

Retrying in 1 seconds...
Retrying in 2 seconds...
Status: 1


['SICXICX', 'SICXICD']


In [784]:
# Cell 45

to_token = contracts['sicx']['SCORE']
params_data = "{\"method\": \"_swap\", \"params\": {\"toToken\":\"" + str(to_token) + "\", \"maxSlippage\":10}}"
data = params_data.encode("utf-8")
params = {'_to': contracts['dex']['SCORE'], '_value': (int) (1 * ICX / 100), '_data': data}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['icd']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

Retrying in 1 seconds...
Retrying in 2 seconds...
Status: 0
Failure: {'code': 32, 'message': 'Passed Maximum slippage'}


In [785]:
# Cell 45

to_token = contracts['icd']['SCORE']
params_data = "{\"method\": \"_swap\", \"params\": {\"toToken\":\"" + str(to_token) + "\", \"maxSlippage\":250}}"
data = params_data.encode("utf-8")
params = {'_to': contracts['dex']['SCORE'], '_value': (int) (1 * ICX / 100), '_data': data}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['sicx']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

Retrying in 1 seconds...
Status: 1
{'scoreAddress': 'cx14a00b375511d6bb1ad4e2c8eee41f4c8a167095', 'indexed': ['Transfer(Address,Address,int,bytes)', 'cx4c73961fd85f13e822f72c2c96120c99e2e637d5', 'hxe7af5fcfd8dfc67530a01a0e403882687528dfcb', '0x23122667c62b00'], 'data': ['0x4e6f6e65']} 

{'scoreAddress': 'cxc60ef756b875786c9f820313ac4015e45726b47d', 'indexed': ['Transfer(Address,Address,int,bytes)', 'cx4c73961fd85f13e822f72c2c96120c99e2e637d5', 'cx354c0ce44e543ebce9478bf0de3621c95473fc90', '0xda475abf000'], 'data': ['0x4e6f6e65']} 

{'scoreAddress': 'cx4c73961fd85f13e822f72c2c96120c99e2e637d5', 'indexed': ['Swap(Address,Address,Address,Address,int,int,int,int)', 'cxc60ef756b875786c9f820313ac4015e45726b47d', 'cx14a00b375511d6bb1ad4e2c8eee41f4c8a167095'], 'data': ['hxe7af5fcfd8dfc67530a01a0e403882687528dfcb', 'hxe7af5fcfd8dfc67530a01a0e403882687528dfcb', '0x2386f26fc10000', '0x23122667c62b00', '0xda475abf000', '0xda475abf000']} 

{'scoreAddress': 'cxc60ef756b875786c9f820313ac4015e45726b47

In [786]:
# Cell 46

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getPoolTotal')\
                    .params({'_pid': 2, '_token': contracts['sicx']['SCORE']}) \
                    .build()
result = icon_service.call(call)
int(result, 0) / 10**18

1.009985

In [787]:
# Cell 47

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getPoolTotal')\
                    .params({'_pid': 2, '_token': contracts['icd']['SCORE']}) \
                    .build()
result = icon_service.call(call)
int(result, 0) / 10**18

0.9901284196560295

In [788]:
# Cell 48

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('totalSupply')\
                    .params({'_pid': 2}) \
                    .build()
result = icon_service.call(call)
int(result, 0) / 10**18

100.0

In [789]:
# Cell 49

transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['dex']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("remove")\
    .params({'_pid':2, '_value': 1 * ICX, '_withdraw': 1}) \
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')
print('\n')

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getPoolTotal')\
                    .params({'_pid': 2, '_token': contracts['icd']['SCORE']}) \
                    .build()
result = icon_service.call(call)
int(result, 0) / 10**18

Retrying in 1 seconds...
Retrying in 2 seconds...
Status: 1
{'scoreAddress': 'cx4c73961fd85f13e822f72c2c96120c99e2e637d5', 'indexed': ['Remove(int,Address,int)', '0x2', 'hxe7af5fcfd8dfc67530a01a0e403882687528dfcb'], 'data': ['0xde0b6b3a7640000']} 

{'scoreAddress': 'cx4c73961fd85f13e822f72c2c96120c99e2e637d5', 'indexed': ['TransferSingle(Address,Address,Address,int,int)', 'hxe7af5fcfd8dfc67530a01a0e403882687528dfcb', 'hxe7af5fcfd8dfc67530a01a0e403882687528dfcb', 'cxf000000000000000000000000000000000000000'], 'data': ['0x2', '0xde0b6b3a7640000']} 

{'scoreAddress': 'cxc60ef756b875786c9f820313ac4015e45726b47d', 'indexed': ['Transfer(Address,Address,int,bytes)', 'cx4c73961fd85f13e822f72c2c96120c99e2e637d5', 'hxe7af5fcfd8dfc67530a01a0e403882687528dfcb', '0x2385e707d1c300'], 'data': ['0x4e6f6e65']} 

{'scoreAddress': 'cx4c73961fd85f13e822f72c2c96120c99e2e637d5', 'indexed': ['Withdraw(Address,Address,int)', 'cxc60ef756b875786c9f820313ac4015e45726b47d', 'hxe7af5fcfd8dfc67530a01a0e403882687528

0.9802271354594692

In [790]:
# Cell 50

data = "{\"method\": \"_deposit\"}".encode("utf-8")
params = {'_to': contracts['dex']['SCORE'], '_value': 30 * ICX, '_data': data}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['icd']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

Retrying in 1 seconds...
Status: 1
{'scoreAddress': 'cx4c73961fd85f13e822f72c2c96120c99e2e637d5', 'indexed': ['Deposit(Address,Address,int)', 'cx14a00b375511d6bb1ad4e2c8eee41f4c8a167095', 'hxe7af5fcfd8dfc67530a01a0e403882687528dfcb'], 'data': ['0x1a055690d9db80000']} 

{'scoreAddress': 'cx14a00b375511d6bb1ad4e2c8eee41f4c8a167095', 'indexed': ['Transfer(Address,Address,int,bytes)', 'hxe7af5fcfd8dfc67530a01a0e403882687528dfcb', 'cx4c73961fd85f13e822f72c2c96120c99e2e637d5', '0x1a055690d9db80000'], 'data': ['0x7b226d6574686f64223a20225f6465706f736974227d']} 



In [791]:
# Cell 51

transaction = TransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['dex']['SCORE'])\
    .value(1 * ICX)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(101)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

Retrying in 1 seconds...
Retrying in 2 seconds...
Status: 1


In [792]:
# Cell 52

transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['dex']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("cancelSicxicxOrder")\
    .params({}) \
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

Retrying in 1 seconds...
Status: 0
Failure: {'code': 32, 'message': 'Withdraw lock not expired'}


In [793]:
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['dex']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("setMarketName")\
    .params({'_pid':3, '_name': 'BALNICD'}) \
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')
print('\n')

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getNamedPools')\
                    .build()
result = icon_service.call(call)
print(result)

Retrying in 1 seconds...
Retrying in 2 seconds...
Status: 1


['SICXICX', 'SICXICD', 'BALNICD']


In [794]:
to_token = contracts['bal']['SCORE']
params_data = "{\"method\": \"_swap\", \"params\": {\"toToken\":\"" + str(to_token) + "\", \"maxSlippage\":250}}"
data = params_data.encode("utf-8")
params = {'_to': contracts['dex']['SCORE'], '_value': (int) (1 * ICX), '_data': data}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['icd']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

Retrying in 1 seconds...
Status: 0
Failure: {'code': 32, 'message': 'Pool does not exist'}
